In [1]:
import sys
import cobra
import cobrakbase
from modelseedpy import KBaseMediaPkg
from modelseedpy import FBAHelper, MSBuilder, MSGenome
from modelseedpy import RastClient

cobrakbase 0.2.8


In [2]:
genome = MSGenome.from_fasta('GCF_000005845.2_ASM584v2_protein.faa', split=' ')

In [3]:
rast = RastClient()

In [4]:
rast.annotate_genome(genome)

[{'execution_time': 1627675119.95746,
  'tool_name': 'kmer_search',
  'hostname': 'pear',
  'parameters': ['-a',
   '-g',
   200,
   '-m',
   5,
   '-d',
   '/opt/patric-common/data/kmer_metadata_v2',
   '-u',
   'http://pear.mcs.anl.gov:6100/query'],
  'id': '88DA5DF4-F170-11EB-AE5A-A9E5BBF382BD'},
 {'execution_time': 1627675120.17851,
  'tool_name': 'KmerAnnotationByFigfam',
  'hostname': 'pear',
  'parameters': ['annotate_hypothetical_only=1',
   'dataset_name=Release70',
   'kmer_size=8'],
  'id': '88FC189A-F170-11EB-AE5A-A9E5BBF382BD'},
 {'parameters': [],
  'id': '892FA53E-F170-11EB-8CA7-18C8BCF382BD',
  'execute_time': 1627675120.51629,
  'tool_name': 'annotate_proteins_similarity',
  'hostname': 'pear'}]

In [5]:
from modelseedpy import MSBuilder
from modelseedpy.helpers import get_template, get_classifier

In [23]:
genome_classifier = get_classifier('knn_filter')

In [8]:
genome_class = genome_classifier.classify(genome)
print(genome_class)

N


In [61]:
from cobrakbase.core.kbasefba.newmodeltemplate_builder import NewModelTemplateBuilder
template_core = get_template('template_core')
template_gramneg = get_template('template_gram_neg')
template_core = NewModelTemplateBuilder.from_dict(template_core, None).build()
template_gramneg = NewModelTemplateBuilder.from_dict(template_gramneg, None).build()
template_core

In [ ]:
kbase = cobrakbase.KBaseAPI()
media1 = kbase.get_from_ws('Carbon-D-Glucose', 'KBaseMedia')
media2 = kbase.get_from_ws('C-acetate', 'KBaseMedia')

In [85]:
model_builder = MSBuilder(genome, template_gramneg)

In [142]:
base_model = model_builder.build('model', '0', allow_all_non_grp_reactions=True, annotate_with_rast=False)
from modelseedpy.core.msbuilder import build_biomass, core_atp
base_model.add_reactions([build_biomass('bio2', base_model, template_core, core_atp, '0')])

In [150]:
def disable_noncore_reactions(ms_atp,model,coretemplate = None):
    if coretemplate == None:
        coretemplate = ms_atp.default_coretemplate
    original_bounds = {}
    noncore_reactions = []
    for reaction in model.reactions:
        if reaction.id[:-1] not in coretemplate.reactions:
            original_bounds[reaction.id] = [reaction.lower_bound,reaction.upper_bound]
            if reaction.lower_bound < 0:
                noncore_reactions.append([reaction,"<"])
            if reaction.upper_bound > 0:
                noncore_reactions.append([reaction,">"])
            reaction.lower_bound = 0
            reaction.upper_bound = 0
            reaction.update_variable_bounds()
    return (noncore_reactions,original_bounds)
noncore_reactions, original_bounds = disable_noncore_reactions(ms_atp, base_model, template_core)

In [166]:
base_model.medium

{'EX_cpd00067_e0': 100,
 'EX_cpd00971_e0': 100,
 'EX_cpd00001_e0': 100,
 'EX_cpd00063_e0': 100,
 'EX_cpd00058_e0': 100,
 'EX_cpd00013_e0': 100,
 'EX_cpd00244_e0': 100,
 'EX_cpd00205_e0': 100,
 'EX_cpd00009_e0': 100,
 'EX_cpd00034_e0': 100,
 'EX_cpd10516_e0': 100,
 'EX_cpd10515_e0': 100,
 'EX_cpd00048_e0': 100,
 'EX_cpd00030_e0': 100,
 'EX_cpd00149_e0': 100,
 'EX_cpd00099_e0': 100,
 'EX_cpd00027_e0': 5,
 'EX_cpd00254_e0': 100,
 'EX_cpd00007_e0': 10}

In [153]:
media_gapfill_stats = ms_atp.evaluate_growth_media(base_model,[media1],'bio2')

/usr/local/lib/python3.8/site-packages/cobra/util/solver.py:508 UserWarning: Solver status is 'infeasible'.


In [158]:
base_model.objective = 'bio2'
pkgmgr = MSPackageManager.get_pkg_mgr(model)
pkgmgr.getpkg("KBaseMediaPkg").build_package(media1)

In [159]:
base_model.medium

{}

In [ ]:
from modelseedpy import MSATPCorrection

In [131]:
ms_atp = MSATPCorrection(template_core)
ms_atp.msgapfill.default_gapfill_templates.append(template_core)

In [132]:
noncore_reactions, original_bounds = ms_atp.disable_noncore_reactions(base_model, template_core)
def evaluate_growth_media2(ms_atp,model, test_medias = None, atp_objective = "bio2"):
    media_gapfill_stats = {}
    with model:
        FBAHelper.set_objective_from_target_reaction(model,atp_objective)
        pkgmgr = MSPackageManager.get_pkg_mgr(model)
        for media in test_medias:
            pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
            solution = model.optimize()
            media_gapfill_stats[media] = None
            print(solution.objective_value)
            if solution.objective_value == 0:
                print('run gapfill', media)
                media_gapfill_stats[media] = ms_atp.msgapfill.run_gapfilling(model, media, atp_objective)
    return media_gapfill_stats
media_gapfill_stats = evaluate_growth_media2(ms_atp, base_model, [media1], 'bio2')

0.0
run gapfill <cobrakbase.core.kbasebiochem.media.Media object at 0x7fbc5c089be0>


/usr/local/lib/python3.8/site-packages/cobra/util/solver.py:508 UserWarning: Solver status is 'infeasible'.


In [133]:
media_gapfill_stats

{<cobrakbase.core.kbasebiochem.media.Media at 0x7fbc5c089be0>: {'reversed': {},
  'new': {}}}

In [134]:
ms_atp.msgapfill.run_gapfilling(base_model, media1, 'bio2')

/usr/local/lib/python3.8/site-packages/cobra/util/solver.py:508 UserWarning: Solver status is 'infeasible'.


{'reversed': {}, 'new': {}}

In [135]:
from modelseedpy.core import FBAHelper, MSGapfill

In [136]:
gapfill = MSGapfill()
gapfill.default_gapfill_templates.append(template_gramneg)

In [137]:
gapfill.run_gapfilling(base_model, media1, 'bio2')

/usr/local/lib/python3.8/site-packages/cobra/util/solver.py:508 UserWarning: Solver status is 'infeasible'.


{'reversed': {}, 'new': {}}

/usr/local/lib/python3.8/site-packages/cobra/util/solver.py:508 UserWarning: Solver status is 'infeasible'.


In [103]:
media_gapfill_stats

{<cobrakbase.core.kbasebiochem.media.Media at 0x7fbc5c089be0>: {'reversed': {},
  'new': {}}}

In [107]:
media_list = ms_atp.determine_growth_media(base_model, media_gapfill_stats, ms_atp.max_gapfilling, 0)
media_list

In [114]:
from modelseedpy.fbapkg.mspackagemanager import MSPackageManager
gapfilling_tests = []
pkgmgr = MSPackageManager.get_pkg_mgr(model)
for media in media_list:
    pkgmgr.getpkg("KBaseMediaPkg").build_package(media)    
    solution = model.optimize()
    print(solution)
    gapfilling_tests.append({
        "media":media,
        "is_max_threshold": True,
        "threshold": 1.2 * solution.objective_value
    })
#Extending model with noncore reactions while retaining ATP accuracy
#filtered = FBAHelper.reaction_expansion_test(model, noncore_reactions, gapfilling_tests)

<Solution 0.000 at 0x7fbc574cb910>


In [113]:
gapfilling_tests

[{'media': <cobrakbase.core.kbasebiochem.media.Media at 0x7fbc5c089be0>,
  'is_max_threshold': True,
  'threshold': 0.0}]

In [108]:
model = ms_atp.apply_growth_media_gapfilling(base_model, media_list, media_gapfill_stats)

In [111]:
    def reaction_expansion_test(model, reaction_list, condition_list):
        # First knockout all reactions in the input list and save original bounds
        original_bound = []
        for item in reaction_list:
            if item[1] == ">":
                original_bound.append(item[0].upper_bound)
                item[0].upper_bound = 0
            else:
                original_bound.append(-1*item[0].lower_bound)
                item[0].lower_bound = 0
        # Now restore reactions one at a time
        count = 0
        filtered_list = []
        for item in reaction_list:
            if item[1] == ">" :
                item[0].upper_bound = original_bound[count]
                if not FBAHelper.test_condition_list(model,condition_list):
                    item[0].upper_bound = 0
                    filtered_list.append(item)
            else:
                item[0].lower_bound = original_bound[count]
                if not FBAHelper.test_condition_list(model,condition_list):
                    item[0].lower_bound = 0
                    filtered_list.append(item)
            count += 1
        return filtered_list

In [110]:
model = ms_atp.expand_model_to_genome_scale(base_model, media_list, noncore_reactions)

KeyError: 'objective'

In [84]:
ms_atp.run_atp_correction(base_model, [media1, media2])

/usr/local/lib/python3.8/site-packages/cobra/util/solver.py:508 UserWarning: Solver status is 'infeasible'.


KeyError: 'objective'

In [18]:
atp_medias = []

In [ ]:
ms_atp.atp_correction(base_model, template_core, atp_medias, 'bio2', genome_class)

In [ ]:
ms_atp.atp_correction(base_model, template_core, atp_medias, 'bio2', genome_class)

In [2]:
#You need a KBase token to use KBase API
#You can get this by running the following code in a code cell in a KBase narrative:

#    import os
#    print "token:"+os.environ['KB_AUTH_TOKEN']

#You can create the API object directly with this token: kbase_api = cobrakbase.KBaseAPI(token)
#Or you can stash the token in the file ~/.kbase/token to avoid putting it in a notebook and use the code below:

kbase_api = cobrakbase.KBaseAPI()
genome = MSGenome.from_fasta('GCF_000005845.2_ASM584v2_protein.faa', split=' ')
print('Number of features:', len(genome.features))
template = kbase_api.get_from_ws("GramNegModelTemplateV3","NewKBaseModelTemplates")
#template = kbase_api.get_from_ws("GramPosModelTemplateV3","NewKBaseModelTemplates")

#The first argument is the model ID and can be anything you want
model = MSBuilder.build_metabolic_model('ecoli', genome, template, allow_all_non_grp_reactions=True)

#Pulling a KBase media to specify an environment for gapfilling
media = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")

#Gapfilling the model in the specified media
model = MSBuilder.gapfill_model(model,"bio1",template,media)

#Applying the same media to the resulting model
kmp = KBaseMediaPkg(model)
kmp.build_package(media)

#Running FBA to demonstrate growth
sol=model.optimize()
model.summary()

Number of features: 4285


Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00009_e0,EX_cpd00009_e0,0.6188,0,0.00%
cpd00013_e0,EX_cpd00013_e0,6.537,0,0.00%
cpd00027_e0,EX_cpd00027_e0,5,6,100.00%
cpd00030_e0,EX_cpd00030_e0,0.002137,0,0.00%
cpd00034_e0,EX_cpd00034_e0,0.002137,0,0.00%
cpd00048_e0,EX_cpd00048_e0,0.1724,0,0.00%
cpd00058_e0,EX_cpd00058_e0,0.002137,0,0.00%
cpd00063_e0,EX_cpd00063_e0,0.002137,0,0.00%
cpd00067_e0,EX_cpd00067_e0,6.213,0,0.00%
cpd00099_e0,EX_cpd00099_e0,0.002137,0,0.00%
